In [28]:
import pandas as pd
import pickle
from pathlib import Path

from lib.util_plot import *
from lib.constants import *

from lib.pandas_impl import *
from lib.pandas_stats_impl import *
from lib.pandas_stats_FLY import FLYAggregateStatistics
from lib.multilevel_analysis import MultilevelAnalysis
from models import *

### Prepare data

In [29]:
data_folder = Path.cwd() / 'data' / 'FLY'
eval_folder = Path.cwd() / 'data' / 'eval' / 'FLY_24-12-17_25mu' # '_10mu' '_25mu'
eval_folder.mkdir(parents=True, exist_ok=True)
plot_folder = eval_folder / "plots"
plot_folder.mkdir(parents=True, exist_ok=True)
synapse_file = data_folder / 'synapses_grid-25000_aggregated.csv' # '_grid-10000_' '_grid-25000_'

In [30]:
df_synapses = pd.read_csv(synapse_file)

# filter self connections
df_synapses = df_synapses[(df_synapses.pre_id_mapped == -1) | (df_synapses.pre_id_mapped != df_synapses.post_id_mapped)]
df_synapses.reset_index(drop=True, inplace=True)

df_synapses_indexed = df_synapses.set_index(["pre_celltype", "post_celltype",  \
                                             "pre_id_mapped", "post_id_mapped", "overlap_volume"])

df_synapses_indexed.sort_index(inplace=True)
df_synapses_indexed.synapse_count.sum()

34156320

Load or initialize local overlap volumes and global index data

In [31]:
index_data = compile_index_data(eval_folder, df_synapses_indexed, recompute=True)

### Model definition

In [32]:
model_P_impl = Model(index_data, ["pre_celltype", "post_celltype"])
model_C_impl = Model(index_data, ["pre_id_mapped", "post_id_mapped"])

# cellular-resolution models 
#model_C_impl = Model(index_data, ["pre_id_mapped", "post_id_mapped"])
#model_Ca_impl = Model(index_data, ["pre_id_mapped"])
#model_Cb_impl = Model(index_data, ["post_id_mapped"])

#model_Pa_impl = Model(index_data, ["post_celltype", "pre_id_mapped"])
#model_PSa_impl = Model(index_data, ["post_celltype", "post_compartment", "pre_id_mapped"])
#model_Sb_impl = Model(index_data, ["post_compartment", "post_id_mapped"])

### Run models

In [33]:
statistics = FLYAggregateStatistics(index_data, compute_syncounts=False, compute_motifs=False, compute_clusters=True)
multilevel_analysis = MultilevelAnalysis(index_data, df_synapses_indexed, statistics, num_realizations=10)
multilevel_analysis.run_null_and_empirical()

  0%|          | 0/10 [00:00<?, ?it/s]

In [34]:
multilevel_analysis.run_model(
    MODEL_NULL,
    model_P_impl,
    MODEL_P
)

  0%|          | 0/10 [00:00<?, ?it/s]

In [35]:
multilevel_analysis.run_model(
    MODEL_NULL,
    model_C_impl,
    MODEL_C
)

  0%|          | 0/10 [00:00<?, ?it/s]

In [36]:
filename = eval_folder / "multilevel_analysis.pkl"
with open(filename, 'wb') as file:
    pickle.dump(multilevel_analysis, file)
